# Gym environment and Implementing Reinforcement Learning Agents 


## 1- Introduction

The **Bipedal Walker** environment is part of the Gymnasium library and is a fascinating and challenging problem in the field of reinforcement learning. 

It requires the control of a simulated robot with **four motorized joints**, guiding it to walk efficiently across uneven terrains. This project focuses on leveraging reinforcement learning techniques to train an agent capable of mastering this environment.  


### Why This Project?  
The decision to work on the Bipedal Walker environment was driven by several key motivations:  
1. **Real-World Relevance**:  
   The principles behind this task are directly applicable to real-world robotics, especially in the domains of bipedal locomotion and autonomous navigation over rough terrains. 

2. **Challenge of Continuous Control**:  
   Unlike simple discrete-action environments, the Bipedal Walker requires handling a **continuous action space**, where precise motor control is essential for success. This adds complexity and provides an excellent opportunity to explore advanced reinforcement learning techniques.  

3. **Rich Dynamics**:  
   The environment simulates realistic physical dynamics, including gravity, friction, and inertia. 


## escolher dentro destes tópicos o melhor

We believe the challenges in this project will revolve around the inherent complexity of the Bipedal Walker environment and the demands of training an effective reinforcement learning agent.
One major challenge will be efficient exploration. The environment requires precise control of the robot's motors to balance, walk, and navigate uneven terrains. Early episodes are likely to result in the robot falling frequently, making it difficult for the agent to explore and learn effectively.

Another significant difficulty is the presence of sparse rewards. The agent is mainly rewarded for moving forward, which means the reward signals are infrequent. This can hinder the learning process, particularly in the early stages. Designing an effective reward strategy without introducing bias will be crucial.

The continuous action space adds another layer of complexity. Unlike discrete-action environments, where the agent selects actions from a finite set, the Bipedal Walker requires precise motor control through continuous values. Solving this problem will require advanced algorithms like Proximal Policy Optimization (PPO) or Deep Deterministic Policy Gradient (DDPG).
Additionally, the physics-based dynamics of the environment introduce realistic but challenging factors, such as gravity, friction, and inertia. Small control errors can lead to cascading failures, such as the robot falling or losing momentum. This makes the task more sensitive to precise tuning and optimization.

The computational demands of the project should not be underestimated. Simulating the environment and training a reinforcement learning agent often requires significant computational power. Achieving consistent results might involve running multiple experiments to tune hyperparameters, further increasing the computational cost.



### reward 

Reward is given for moving forward, total 300+ points up to the far end. If the robot falls, it gets -100. Applying motor torque costs a small amount of points, more optimal agent will get better score. State consists of hull angle speed, angular velocity, horizontal speed, vertical speed, position of joints and joints angular speed, legs contact with ground, and 10 lidar rangefinder measurements. There's no coordinates in the state vector.

![Descrição da imagem](./images/bipedal_scheme.png)

### Imports 

In [29]:
from pyvirtualdisplay import Display
import gymnasium as gym
from stable_baselines3 import PPO

### Start Virtual Display 

Please pay attention to the operative system you are using when running this code -- mudar isto? nem sei se está a rodar 

In [ ]:
#start virtual display - WINDOWS
display = Display(visible=0, size=(1024, 768))
display.start()
import os
os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)

In [30]:
#start virtual display - MAC
env = gym.make('CartPole-v1')
env.reset()
env.render()  # Aqui, o ambiente será renderizado diretamente no macOS


### Observation Space

Our agent should fit to whatever it is, but it's better to know the inputs if our agent is not learning. 

Here is the observation table from [this link](https://github.com/openai/gym/wiki/BipedalWalker-v2), with 24 different parameters in one state:


In [33]:
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample()) # Get a random observation

Observation Space Shape (24,)
Sample observation [ 2.7958372   4.2719393  -3.098417    3.5200021  -1.5043272   0.17990619
 -1.7329019  -2.693569    0.8306578   2.8606198   3.5075529  -1.6949596
  4.8863325   3.932315   -0.344987    0.8760876   0.79466033 -0.07690509
 -0.40360272 -0.35994315  0.80476236 -0.15696786  0.04681255  0.05192911]


![Descrição da imagem](./images/observation_space.png)



According to [this link](https://colab.research.google.com/github/BrutFab/ppo_BipedalWalker_v3/blob/main/ppo_BipedalWalker_v3.ipynb#scrollTo=xr8LRvHOLxIx),Observation Space is a vector of size 24 and each value contains different information about the walker: 


- **Hull Angle Speed**: The speed at which the main body of the walker is rotating.
- **Angular Velocity**: The rate of change of the angular position of the walker.
- **Horizontal Speed**: The speed at which the walker is moving horizontally.
- **Vertical Speed**: The speed at which the walker is moving vertically.
- **Position of Joints**: The positions (angles) of the walker's joints. Given that the walker has 4 joints, this take up 4 values.
- **Joints Angular Speed**: The rate of change of the angular position for each joint. Again, this would be 4 values for the 4 joints.
- **Legs Contact with Ground**: Indicating whether each leg is in contact with the ground. Given two legs, this contains 2 values.
- **10 Lidar Rangefinder Measurements**: These are distance measurements to detect obstacles or terrain features around the walker. There are 10 of these values.


### Action Space 

BipedalWalker has two legs. Each leg has two joints. You have to teach the Bipedal-walker to walk by applying the torque on these joints.

 Therefore the size of our action space is four which is the torque applied on four joints. 

 Actions are motor speed values in the [-1, 1] range for each of the 4 joints at both hips and knees.

In [19]:
print("Action Space Shape", env.action_space.shape)
print("Action Space Sample", env.action_space.sample()) 

Action Space Shape (4,)
Action Space Sample [-0.43058464  0.9464465  -0.2230738  -0.30005386]


![Descrição da imagem](./images/action_space.png)

### Setup Environment 

In [31]:
env_id = "BipedalWalker-v3"
env = gym.make(env_id, hardcore=True)

### Vectorized Environment

> **Vectorized Environments** are a method for stacking multiple independent environments into a single environment. Instead of training an RL agent on 1 environment per step, it allows us to train it on n environments per step. Because of this, actions passed to the environment are now a vector (of dimension n). It is the same for observations, rewards and end of episode signals (dones). In the case of non-array observation spaces such as Dict or Tuple, where different sub-spaces may have different shapes, the sub-observations are vectors (of dimension n).

*This definition was taken from the official Stable Baselines 3 documentation. For more details, visit the [official guide on vectorized environments](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html).*



In [34]:
from stable_baselines3.common.env_util import make_vec_env
env = make_vec_env('BipedalWalker-v3', n_envs=16)

We chose to use 16 environments in parallel (n_envs=16) to speed up the training process.

In [32]:
model = PPO(
    policy = 'MlpPolicy',
    env = env,
    n_steps = 2048,
    batch_size = 128,
    n_epochs = 6,
    gamma = 0.999,
    gae_lambda = 0.98,
    ent_coef = 0.01,
    verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
